# Imports

In [1]:
from IPython.display import display as dsp
from datetime import datetime as dt
from random import sample


import sys
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
from canalyst_candas.configuration.config import Config

config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}

config = Config(config=config_info)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



A configuration file has been created for you in 
/home/canalyst-candas/canalyst/keys.json. For Excel file downloads and scenario 
mapping, you'll need to add an API key to the 'canalyst_api_key' 
value. Visit https://app.canalyst.com/settings/api-tokens to create 
and retrieve your API key. You will also need to retrieve your 
S3 Access ID and Secret Key and fill in 'canalyst_s3_id' and 
'canalyst_s3_key' with those values, respectively. If using a 
Jupyter Notebook, stop and restart the notebook for the changes to 
take effect. If using a Python/iPython session, quit the current 
session and start a new one.


# Establish an Interest List

In [2]:

ir_list = [
"AC CN",
"ALK US",
"ALGT US",
"AAL US",
"CPA US",
"DAL US",
"ULCC US",
"HA US",
"JBLU US",
"LUV US",
"SAVE US",
"UAL US",
]


# Creating A Search Object

In [3]:
# Instantiate new search object:
search = cd.Search(config)


# Searching for Common Time Series Which Are Drivers

In [4]:
search_results_df = search.search_time_series(
    ticker = ir_list,
    time_series_name="",
    is_driver=True   
)
# Common time series:
gpd_search = search_results_df.groupby("time_series_name")[["ticker"]].nunique().reset_index().copy()
gpd_search.sort_values(by="ticker", ascending=False, inplace=True)
gpd_search.head(20)

Building search index...
No.of.unique tickers:  12
No.of.unique time series:  311


,time_series_name,ticker
53,MO_RIS_TaxRate_Deferred,12
48,MO_RIS_OI,12
4,MO_CFSum_Acquisition,12
5,MO_CFSum_Capex,12
6,MO_CFSum_DPS,12
7,MO_CFSum_Divestiture,12
52,MO_RIS_TaxRate_Current,12
42,MO_RIS_Adjustments_NONGAAP,11
8,MO_CFSum_NetDebtIssuance,11
9,MO_CFSum_NetShares,11


In [5]:
search_results_df = search.search_time_series(
    ticker = ir_list,
    time_series_name="eps",
    mo_only=True
)
# Common time series:
gpd_search = search_results_df.groupby("time_series_name")[["ticker"]].nunique().reset_index().copy()
gpd_search.sort_values(by="ticker", ascending=False, inplace=True)
gpd_search.head(10)

No.of.unique tickers:  12
No.of.unique time series:  3


,time_series_name,ticker
0,MO_RIS_EPS_WAB,12
1,MO_RIS_EPS_WAD,12
2,MO_RIS_EPS_WAD_Adj,12


In [6]:
ts = "MO_BSS_InterestRate_Debt"
search_results_df = search.search_time_series(
    ticker=ir_list,
    time_series_name=ts,
    unit_type='percentage',
    is_driver=True
)

No.of.unique tickers:  10
No.of.unique time series:  1


In [7]:
# A list of tickers for this driver:
ir_tickers = list(search_results_df[search_results_df["time_series_name"] == ts]["ticker"].unique())
len(ir_tickers)

10

# Create a Model Set of Target Companies

In [8]:
# Create a model set form this ticker list:

ir_set = cd.ModelSet(ticker_list=ir_tickers, config=config, allow_nulls=True)


Please instantiate ModelSet with a list of tickers, even a list of one.


In [35]:
# Create params:
df_params = ir_set.forecast_frame(ts,
                             n_periods=4,
                             function_name='add',
                             function_value=1)
df_params = df_params[df_params['value']!=0]
dsp(df_params.shape)
dsp(df_params.head())

(40, 7)

,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
3284,JBLU US,Q1-2022,MO_BSS_InterestRate_Debt,Effective Interest Rate on Debt,4.10000,5.10000,2022-03-31
3677,LUV US,Q1-2022,MO_BSS_InterestRate_Debt,Effective Interest Rate on Debt,4.00000,5.00000,2022-03-31
3703,SAVE US,Q1-2022,MO_BSS_InterestRate_Debt,Effective Interest Rate on Debt,4.50000,5.50000,2022-03-31
3785,HA US,Q1-2022,MO_BSS_InterestRate_Debt,Effective Interest Rate on Debt,5.60000,6.60000,2022-03-31
4052,ALGT US,Q1-2022,MO_BSS_InterestRate_Debt,Effective Interest Rate on Debt,4.20000,5.20000,2022-03-31


In [10]:
sample_tickers = list(df_params["ticker"].unique())
sample_params = df_params[df_params["ticker"].isin(sample_tickers)].copy()
len(sample_tickers)

10

# Run Scenarios on Common Time Series

In [11]:
start = dt.now()

# How much does revenue move?
return_series = "MO_RIS_EPS_WAD_Adj"

# Scenarios for a sample of 10:
dicts_output=ir_set.fit(sample_params,return_series)

# Print time:
dsp((dt.now() - start).seconds)

ALK US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/E82OUG0135/equity-models/FY2021.20/scenarios/QTV1F6KpU5G2c4DMWYzjkg/
UAL US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/K25QNC0129/equity-models/FY2021.23/scenarios/iWcKpgBZUnG-Xd3KbRo3bg/
AAL US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/DYKK2Y0171/equity-models/FY2021.20/scenarios/1lZ2EtDpW8Kq7mYbyVl9Iw/
LUV US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/SAQ7HK0156/equity-models/FY2021.20/scenarios/4Er-G-wKU92erEvDhj_0LQ/
HA US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/XGJ4AE0144/equity-models/FY2021.20/scenarios/LqyXDwzIXXKZp1FKykTikg/
AC CN scenario_id_url: https://mds.canalyst.com/api/equity-model-series/9O09KW0128/equity-models/Q3-2021.21/scenarios/0TUwyGcRVQ66kSSDkD91Rw/
DAL US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/2EFOXB0147/equity-models/FY2021.20/scenarios/3yjqKeEdXGGJNohQDd7M2g/
ALGT US

60

# Rank Ordered Results

In [12]:
# Show scenario results:
list_df = []
for k in dicts_output.keys():
    df = dicts_output[k].head(1)
    list_df.append(df)
pd.concat(list_df).sort_values('diff')

,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,SAVE US,Q4-2021,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-1.56106,-1.5610569492,1.00000
0,UAL US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-5.20966,-5.2096557999,1.00000
0,ALGT US,Q4-2021,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,2.62148,2.6214776886,1.00000
0,AAL US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-1.35021,-1.3502134297,1.00000
0,AC CN,Q4-2021,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-1.27607,-1.2760685093,1.00000
0,JBLU US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-0.59935,-0.9604456026,1.60247
0,LUV US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-0.44975,-0.7692580813,1.71042
0,HA US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-1.30715,-3.2281527603,2.46960
0,DAL US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-0.40007,-1.1762191937,2.94005
0,ALK US,Q1-2022,MO_RIS_EPS_WAD_Adj,Adjusted Earnings Per Share - WAD,-0.50709,-2.0125171855,3.96875


# Investigate the PNL of one company

In [26]:
ticker = 'ALK US'
time_series_name = "MO_RIS_EPS_WAD_Adj"
target_model_set = cd.ModelSet(ticker_list=[ticker], config=config) 
model_map = target_model_set.create_model_map(ticker=ticker,
                            time_series_name=time_series_name,
                            tree = True,
                            notebook = True,
                            col_for_labels = "time_series_name") 
model_map.show()

https://mds.canalyst.com/api/equity-model-series/E82OUG0135/equity-models/FY2021.20/time-series/MO_RIS_EPS_WAD_Adj/forecast-data-points/Q1-2022/drivers/?format=dot


# Investigate one aspect of a company model, using most recent quarter

In [24]:
test_set = cd.ModelSet(ticker_list=['ALK US'], config=config, allow_nulls=True)
test_set.model_frame(mrq=True,
                     #time_series_name = 'cash|debt',
                     time_series_name='MO_BSS_InterestRate_Debt|cash|debt',
                     category='')[['time_series_description','time_series_name','value']].groupby('time_series_description').first().reset_index()

Please instantiate ModelSet with a list of tickers, even a list of one.


,time_series_description,time_series_name,value
0,Adjusted EBT,z_E82OUG0135_MO_AN_AdjustedEBT,236000000.00000
1,Adjusted Net Debt / LTM EBITDA,MO_BSS_Debt_ToEBITDA,2.01573
2,Adjusted Net Debt / LTM EBITDAR,MO_BSS_Debt_ToEBITDAR,1.27596
3,Adjusted Net Debt / Operating Cash Flow,MO_BSS_Debt_ToCF,0.65000
4,Beginning Cash Balance,MO_CCFS_Balance_Begin,1386000000.00000
5,Cash Flow Per Diluted Share,MO_CFSum_CFPS,2.31940
6,Cash and cash equivalents,MO_BS_Cash,495000000.00000
7,Cash paid for acquisitions,z_E82OUG0135_MO_CCFS_Cashpaidforacquisitions,0.00000
8,"Cash, short-term investments, and hedge margin receivable",MO_BSS_Cash,3219000000.00000
9,Consensus Estimates - Adjusted EBT,z_E82OUG0135_MO_RIS_ConsensusEstimatesAdjustedEBT,NaN
